In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='API_KEY',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.apple.com/en-us/details/200569946/aiml-intern?team=STDNT")
page_data = loader.load().pop().page_content
page_data

"\n\n\n\n\n\n\n\n\nAIML Intern - Careers at Apple\n\n\n\n\n\n\n\nAppleStoreMaciPadiPhoneWatchVisionAirPodsTV & HomeEntertainmentAccessoriesSupport\n\n\n0+\nCareers at AppleOpen MenuClose Menu\n\n      Work at Apple\n    \n \n\n      Life at Apple\n    \n \n\n      Profile\n    \n \n\n      Sign In\n    \n \nSearch\nJobs at Apple\nAIML InternShanghai, Shanghai, ChinaHardwareAdd to Favorites AIML InternRemoved from favoritesAdd a favoriteCloseTo view your favorites, sign in with your Apple ID.Sign InDon’t have an Apple ID?Create one nowForgot your Apple ID or password?Submit ResumeAIML InternBack to search resultsSummaryPosted: Oct 22, 2024Weekly Hours: 40 Role Number:200569946At Apple, our team doesn't just build products; they create the kind of wonder that has revolutionized entire industries. It’s the diversity of our people and their ideas that drives the innovation central to everything we do—from remarkable technology to industry-leading environmental efforts. Join Apple and contr

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "AIML Intern",
  "experience": "Bachelor degree in Computer Science, Automation Engineering, Electrical Engineering, etc.",
  "skills": [
    "Strong programming skills in Python, or a similar language",
    "Experience in data analysis and machine learning, with proficiency in libraries such as TensorFlow, PyTorch, Scikit-Learn, etc.",
    "Solid understanding of statistical methods and algorithms",
    "Excellent problem-solving skills and critical thinking abilities",
    "Strong communication skills, with the ability to explain complex technical concepts clearly and concisely",
    "Experience or coursework in computer vision or natural language processing (NLP) models"
  ],
  "description": "This role provides an exceptional opportunity to gain hands-on experience working on innovative projects that enhance the reliability and performance of Apple's products through advanced AI and ML applications. With access to all necessary resources, you'll push the boundar

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'AIML Intern',
 'experience': 'Bachelor degree in Computer Science, Automation Engineering, Electrical Engineering, etc.',
 'skills': ['Strong programming skills in Python, or a similar language',
  'Experience in data analysis and machine learning, with proficiency in libraries such as TensorFlow, PyTorch, Scikit-Learn, etc.',
  'Solid understanding of statistical methods and algorithms',
  'Excellent problem-solving skills and critical thinking abilities',
  'Strong communication skills, with the ability to explain complex technical concepts clearly and concisely',
  'Experience or coursework in computer vision or natural language processing (NLP) models'],
 'description': "This role provides an exceptional opportunity to gain hands-on experience working on innovative projects that enhance the reliability and performance of Apple's products through advanced AI and ML applications. With access to all necessary resources, you'll push the boundaries of what’s possible and achie

In [ ]:
type(json_res)

dict

In [ ]:
import pandas as pd

df = pd.read_csv("/content/my_portfolio.csv")
df

,Techstack,Links
0,"Python, CNN, matplotlib, tensorflow, keras",https://github.com/Merc-Gonsalves/AI-Accident-...
1,"Python, textblob, newspaper3k, nltk, tkinter",https://github.com/Merc-Gonsalves/News-Summary
2,"Python, mobilenetv2, tensorflow, matplotlib, o...",https://github.com/Merc-Gonsalves/Object-Detec...
3,"Python, langchain, chromadb, gpt",https://github.com/Merc-Gonsalves/RAG-using-la...
4,"Python, langchain, huggingface, arxiv, acceler...",https://github.com/Merc-Gonsalves/Research-pap...
5,"Python, ultralytics, roboflow, yolo, ipython, ...",https://github.com/Merc-Gonsalves/Smoke-Fire-D...
6,"Python, pytorch, trl, peft, datasets, transfor...",https://github.com/Merc-Gonsalves/AI-Doctor-Ch...
7,"Python, Alpaca, backtesting, rest",https://github.com/Merc-Gonsalves/Stock-Trader...
8,"HTML, CSS, javascript, php, react, webstream",https://github.com/Merc-Gonsalves/Final-Project
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [ ]:
job

{'role': 'AIML Intern',
 'experience': 'Bachelor degree in Computer Science, Automation Engineering, Electrical Engineering, etc.',
 'skills': ['Strong programming skills in Python, or a similar language',
  'Experience in data analysis and machine learning, with proficiency in libraries such as TensorFlow, PyTorch, Scikit-Learn, etc.',
  'Solid understanding of statistical methods and algorithms',
  'Excellent problem-solving skills and critical thinking abilities',
  'Strong communication skills, with the ability to explain complex technical concepts clearly and concisely',
  'Experience or coursework in computer vision or natural language processing (NLP) models'],
 'description': "This role provides an exceptional opportunity to gain hands-on experience working on innovative projects that enhance the reliability and performance of Apple's products through advanced AI and ML applications. With access to all necessary resources, you'll push the boundaries of what’s possible and achie

In [ ]:
job = json_res
job['skills']

['Strong programming skills in Python, or a similar language',
 'Experience in data analysis and machine learning, with proficiency in libraries such as TensorFlow, PyTorch, Scikit-Learn, etc.',
 'Solid understanding of statistical methods and algorithms',
 'Excellent problem-solving skills and critical thinking abilities',
 'Strong communication skills, with the ability to explain complex technical concepts clearly and concisely',
 'Experience or coursework in computer vision or natural language processing (NLP) models']

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_desc}

        ### INSTRUCTION:
        You are Merc Gonsalves who is a fresher with a bachelors degree in computer science.
        Here is the github link with all your projects: https://github.com/Merc-Gonsalves
        you have 3 months of experience as an AI intern at Corizo and another 3 months as an AI intern at Hackveda
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
        Remember you are Merc Gonsalves, fresher with a bachelors degree in computer science.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_desc": str(job), "link_list": links})
print(res.content)

Subject: Application for AIML Intern Role at Apple

Dear Hiring Manager,

I am excited to apply for the AIML Intern role at Apple, as advertised. With a strong foundation in computer science and a passion for artificial intelligence and machine learning, I am confident that I can make a valuable contribution to your team.

As a recent graduate in Computer Science, I have developed a solid understanding of statistical methods and algorithms, as well as excellent problem-solving skills and critical thinking abilities. My experience as an AI intern at Corizo and Hackveda has provided me with hands-on experience in data analysis and machine learning, with proficiency in libraries such as TensorFlow, PyTorch, and Scikit-Learn.

I would like to highlight my capabilities in fulfilling your needs through my previous projects, which can be found on my GitHub page: https://github.com/Merc-Gonsalves. Some of my most relevant projects include:

* Machine Learning with Python: https://example.com/m